# This notebook will be mainly used for the capstone project

In [8]:
!conda list 


# packages in environment at C:\Users\gjarantowski\Anaconda3\envs\Ecommerce:
#
# Name                    Version                   Build  Channel
alabaster                 0.7.12                   pypi_0    pypi
anyio                     2.2.0            py38haa95532_2  
argon2-cffi               20.1.0           py38h2bbff1b_1  
astroid                   2.4.2                    py38_0  
async_generator           1.10               pyhd3eb1b0_0  
atomicwrites              1.4.0                      py_0  
attrs                     19.3.0                     py_0    conda-forge
babel                     2.9.1              pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
beautifulsoup4            4.9.3              pyha847dfd_0  
blas                      1.0                         mkl    conda-forge
bleach                    3.3.0              pyhd3eb1b0_0  
brotlipy                  0.7.0           py38h2bbff1b_1003  
bs4                       4.9.3         

In [11]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib


In [15]:
html = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

In [18]:
soup = BeautifulSoup(html)

In [21]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [27]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [31]:
geo.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [32]:
geo.columns = ['PostalCode', 'Latitude', 'Longitude']

In [33]:
df = df.merge(geo, on =['PostalCode'])

In [34]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [36]:
import folium

In [38]:
map_toronto = folium.Map()

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

In [39]:
map_toronto